## Applied Reinforcement Learning - Tutorial - Panda-Gym
### [Armin Niedermueller](https://github.com/nerovalerius)
### Salzburg University of Applied Sciences

### Covered Topics
* Panda-Gym Introduction
    * Franka Panda Robot
    * Setup
* Example Environment - Panda Reach
* Create a custom robot
* Create a custom task with obstacles in the scene
* Custom a custom environment

### Panda-Gym Introduction
Panda-Gym is a reinforcement learning environment for the Franka Emika Panda robot. It is based on the OpenAI Gym framework and provides a set of tasks that can be used to train reinforcement learning agents. The tasks are based on the PyBullet physics engine and can be used to train agents for real-world applications. The environment is designed to be easy to use and extend. It is also possible to create custom robots, environments and tasks.
A detailed documentation can be found [here](https://panda-gym.readthedocs.io/en/latest/) and their paper can be cited as folows:
```
@article{gallouedec2021pandagym,
  title        = {{panda-gym: Open-Source Goal-Conditioned Environments for Robotic Learning}},
  author       = {Gallou{\'e}dec, Quentin and Cazin, Nicolas and Dellandr{\'e}a, Emmanuel and Chen, Liming},
  year         = 2021,
  journal      = {4th Robot Learning Workshop: Self-Supervised and Lifelong Learning at NeurIPS},
}
```

#### Franka Panda Robot
Panda is a collaborative robot with 7 degrees of freedom developed by [FRANKA EMIKA](https://www.franka.de/).
It can be programmed directly with a graphical user interface or with the Robot Operating System 1 & 2 (C++, MoveIt!, Rviz and so on).
The torque sensors on it's 7 seven axes make this robot arm so sensitive, that it even stops at a balloon.
It works at a very high precision as well as stability, which makes it a perfect tool for research and development.

<img src="images/franka_panda.png"  width="35%"> \
Image source: [LINK](https://github.com/nerovalerius/collision_avoidance/blob/master/BAC2_niedermueller.pdf)

I worked with the Panda robot for my bachelor thesis, where i used two 3D stereo cameras to enable collision avoidance for the robot arm. The robot was able to avoid obstacles while reaching a target. The code and results can be found [here](https://github.com/nerovalerius/collision_avoidance) and [here](https://www.youtube.com/watch?v=LQPS--bnvQY)

#### Setup
Before we are able to start programming, we need to prepare our programming environment.

##### Linux
First of all, we create a virtual environment for our undertaking, in order to avoid conflicts with other projects.
We use the conda package manager to create a virtual environment. If you are not familiar with conda, you can find a tutorial [here](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html).

First, download and install miniconda.
```
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh \
    && bash Miniconda3-latest-Linux-x86_64.sh -b \
    && rm Miniconda3-latest-Linux-x86_64.sh
```
Follow the instructions on the terminal and also initialize conda for your current shell.
```
conda init bash
```
Now, create a new environment and activate it.
```
conda create -n panda-gym-tutorial python=3.9
conda activate panda-gym-tutorial
```

##### Window
1. Install [Anaconda](https://www.anaconda.com/)
2. Install [VS Code](https://code.visualstudio.com/).\
    a. Install [Python Extension](https://marketplace.visualstudio.com/items?itemName=ms-python.python) for VS Code\
    b. Install [Jupyter Extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) for VS Code

For panda-gym, there is currently no conda package available. Therefore, we need to install it with pip.
Furthermore, we can install numpngw to store the rendered images as animated png files.

In [ ]:
%pip install panda-gym
%pip3 install numpngw

### Example Environment - Panda Reach
Panda-Gym defines for each task a separate environment. Let's take a look at the environment for the **Panda Reach** environment, where the robot has to reach a target position with its end-effector:

```
import numpy as np
from panda_gym.envs.core import RobotTaskEnv
from panda_gym.envs.robots.panda import Panda
from panda_gym.envs.tasks.reach import Reach

class PandaReachEnv(RobotTaskEnv):
    """Reach task wih Panda robot.
    Args:
        render (bool, optional): Activate rendering. Defaults to False.
        reward_type (str, optional): "sparse" or "dense". Defaults to "sparse".
        control_type (str, optional): "ee" to control end-effector position or "joints" to control joint values.
            Defaults to "ee".
    """

    def __init__(self, render: bool = False, reward_type: str = "sparse", control_type: str = "ee") -> None:
        # use PyBullet as simulation backend
        sim = PyBullet(render=render)
        # use Panda robot, define its control type and initial position
        robot = Panda(sim, block_gripper=True, base_position=np.array([-0.6, 0.0, 0.0]), control_type=control_type)
        # use Reach task, define its reward type and initial end-effector position
        task = Reach(sim, reward_type=reward_type, get_ee_position=robot.get_ee_position)
        super().__init__(robot, task)
```

We define PyBullet as our physics engine for robotics. It is used to simulate and render the robot and the environment.

The robot is defined in [/panda_gym/envs/robots/panda](https://github.com/qgallouedec/panda-gym/blob/master/panda_gym/envs/robots/panda.py). \
Here, you can find all necessary physics parameters (such as friction) and functions (such as set_action).

Furthermore, the task is defined as Reach task, defined in [panda_gym/envs/tasks/reach.py](https://github.com/qgallouedec/panda-gym/blob/master/panda_gym/envs/tasks/reach.py).\
Inside this file, the 3D environment (such as the table) is defined and the reward is computed. 

Now, three parameters can be set in the environment:
* **render:** activate or deactivate the rendering of the environment
* **reward_type:**
    * sparse: reward is 1 if the target is reached and 0 otherwise
    * dense: reward is the distance between the target and the end-effector
* **control_type:** actions should either control the robot's:
    * end-effector position
    * or joint values

#### Code and Animation
<img src="images/reach.png"  width="35%"> 

First, we create "PandaReach-v3" environment and set render to True, to see what the robot is learning. Then, we reset the environment, define our number of maximum number of episodes and let the robot take actions provided by our policy. After each step, we check if the episode is done and if so, we reset the environment. 


Beside the actual reinforcement learning, we also create an animation of the robot's learning process by storing the rendered images inside a png file.

In [ ]:
###################################
# Task: Panda Reach               #
###################################

import gymnasium as gym
import panda_gym
from tqdm import tqdm
from numpngw import write_apng

# please choose one of the following render modes
render_mode = 'human'       # opens OpenGL window for rendering
#render_mode = 'rgb_array'  # returns RGB image as numpy array, if the render should be stored as animated png

# create environment and activate rendering
if render_mode == 'rgb_array':
    images = [] # array to store images

env = gym.make("PandaReach-v3", render_mode=render_mode)

# define low frame rate for rendering to reduce computational load
env.metadata['render_fps'] = 24

# reset environment and get initial observation (either state of the joints or the end effector position)
observation, info = env.reset()
# define maximum nuber of episodes
max_steps = 10000

# run simulation 
for step in tqdm(range(max_steps)):
    # take action as defined by our policy
    action = env.action_space.sample()
    
    # execute action and get new observation, reward, termination flag and additional info
    observation, reward, terminated, truncated, info = env.step(action)

    # add each image to our array for each step to create an animation afterwards
    if render_mode == 'rgb_array':
        images.append(env.render())

    # when the episode is terminated, reset the environment
    if terminated or truncated:
        observation, info = env.reset()

env.close()

if render_mode == 'rgb_array':
    # save animation
    print("Saving animation...")
    write_apng('images/reach.png', images, delay = 5)
print("finished")

### Create a custom robot
This section follows loosely the [panda-gym tutorial](https://panda-gym.readthedocs.io/en/latest/custom/custom_robot.html) for creating a custom robot. 

#### Create URDF
First, we need to create a [URDF](http://wiki.ros.org/urdf) (Unified Robot Description Format) file for our robot. 
URDF is an XML file format used in ROS to describe all elements of a robot. It is used to define the robot's kinematic and dynamic structure, as well as its physical properties.
Luckily, there are some tools available to visualize URDF files while writing them. For example [URDF Modeler](https://mymodelrobot.appspot.com/5629499534213120), which we will use in this tutorial.

A URDF file consists of several parts:
* **robot:** the root element of the file
* **link:** a rigid body
* **joint:** a joint connecting two links

and some optional parts:
* **material:** defines the color of a link
* **transmission:** defines the transmission between a joint and a motor

We begin by defining the robot's name and the base link. The base link is the link that is fixed to the ground. In our case, we will use the base link to define the robot's initial position.

```
  	<link name="link0"> <!--base_link -->
		<visual>
			<origin xyz="0 0 0" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.1" length="0.05"/>
			</geometry>
	     	<material name="Cyan3">
				<color rgba="0 0.5 0.5 0"/>
			</material>
	   </visual>
	</link>
```

This is a squared base with dimensions 0.3 x 0.3 x 0.05 m. The origin of the visual element is defined by the xyz and rpy attributes. The xyz attribute defines the position of the origin in the link's frame. The rpy attribute defines the orientation of the origin in the link's frame. The orientation is defined by the roll, pitch and yaw angles. The roll angle is the rotation around the x-axis, the pitch angle is the rotation around the y-axis and the yaw angle is the rotation around the z-axis. The roll, pitch and yaw angles are defined in radians. The origin of the link's frame is defined by the xyz and rpy attributes of the link element.

The first joint connects the first arm part to the base_link, the type is revolute, which means that the joint is a hinge joint.

```
	<joint name="joint0" type="revolute">  <!-- base -->
    	<parent link="link0"/>
    	<child link="link1"/>
    	<origin xyz="0 0 0.10" rpy="0 0 0"/>
    	<axis xyz="0 0 1"/>
    	<limit lower="-0.6" upper="0.6" effort="0" velocity="3"/>
  	</joint>
```

We now have our base defined. For each part of our robot, we define a link for the arm part itself, which consists of a sphere link connected with a fixed joint to an arm link:

```	
  	<link name="link1"> <!-- cylinder0 -->
		<visual>
			<origin xyz="0 0 0" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.05" length="0.2"/>
			</geometry>
	     	<material name="Yellow2">
				<color rgba="0.8 0.8 0 1.0"/>
			</material>
	   </visual>
	</link>	
	
	<link name="link2"> <!-- ball0 -->
		<visual>
			<origin xyz="0 0 0" rpy="0 0 0"/>
			<geometry>
				<sphere radius="0.052"/>
			</geometry>
			<material name="Cyan1">
	       		<color rgba="0 0.9 0.9 1.0"/>
			</material>
	   </visual>
	</link>

 	<joint name="joint1" type="revolute">  <!-- cylinder0 -->
    	<parent link="link1"/>
    	<child link="link2"/>
    	<origin xyz="0 0 0.1" rpy="0 0 0"/>
		<limit lower="0.5" upper="1.6" effort="10" velocity="3"/>
        <axis xyz="0 1 0"/>
	</joint>

```

The sphere created is to connect the links more realistically. Not only the visual geometry is defined, but also the collision geometry, which is used to avoid collision with other parts in the environment and the robot parts itself. Typically, the geometric dimensions are the same as the visual dimensions.

Between the arm links and the spheres of the next arm link, a movable joint is needed, such as:

```
<joint name="joint_2" type="revolute">
    	<parent link="link_1"/>
    	<child link="jointlink_2"/>
    	<origin xyz="0 0 0.1" rpy="0 0 0"/>
		<limit lower="-1.57" upper="1.57" effort="10" velocity="3"/>
        <axis xyz="0 1 0"/>
	</joint>
``` 

We define a revolute joint, with the limits of -pi/2 to pi/2 and rotation around the y-axis. The effort and velocity limits are defined in Nm and rad/s, respectively. The effort limit is the maximum torque that can be applied to the joint. The velocity limit is the maximum angular velocity that can be applied to the joint.

After building each links and joints onto each other, we place a gripper (or: end effector) on the last link of our robot, in order to let the robot grip objects.

```
    <link name="link8"> <!-- gripper_base -->
  ...
    <visual>
      <geometry>
        <box size="0.105 .02 .025"/>
      </geometry>
      <origin rpy="0 0 0" xyz="0 -0.1 0"/>
      <material name="white">
        <color rgba="1 1 1 1"/>
      </material>
    </visual>
    ...
</link>

  <link name="link9"> <!-- left_fingertip -->
    ...
    <visual>
      <geometry>
        <box size="0.025 .05 0.025"/>
      </geometry>
      <origin rpy="0 0 0" xyz="0 0 0"/>
      <material name="white">
        <color rgba="1 1 1 1"/>
      </material>
    </visual>
    ...
  </link>

    <link name="link10"> <!-- right fingertip -->
    ...
    <visual>
      <geometry>
        <box size="0.025 .05 0.025"/>
      </geometry>
      <origin rpy="0 0 0" xyz="0 0 0"/>
      <material name="white">
        <color rgba="1 1 1 1"/>
      </material>
    </visual>
    ...
  </link>
  ```

Our gripper has two fingertips, which can be moved to grip an object. How they move is, again, defined as joint:

```
 	<joint name="joint7" type="revolute">  <!-- gripper_base -->
    	<parent link="link7"/>
    	<child link="link8"/>
    	<origin xyz="0 0 0" rpy="-1.5708 0  0"/>
        <axis xyz="0 1 0"/>
    	<limit lower="-1.57" upper="1.57" effort="10" velocity="3"/>
	</joint>
 
  <joint name="joint8" type="prismatic">  <!-- gripper_base_to_right_fingertip -->
    <axis xyz="1 0 0"/>
    <limit effort="1.0" lower="-0.03" upper="0.0" velocity="0.5"/>
    <parent link="link8"/>
    <child link="link9"/>
    <origin xyz="0.04 -0.135 -0.0"/>
  </joint>

    <joint name="joint9" type="prismatic">  <!-- gripper_base_to_left_fingertip -->
    <axis xyz="1 0 0"/>
    <limit effort="1.0" lower="0" upper="0.03" velocity="0.5"/>
    <parent link="link8"/>
    <child link="link10"/>
    <origin xyz="-0.04 -0.135 0.0"/>
  </joint>
```

Please consider the complete file under [urdf/robot.urdf](urdf/robot.urdf). 
An Animation created with [URDF Modeler](https://mymodelrobot.appspot.com/5629499534213120) of the urdf file shows how our links and joints move:

<img src="images/urdf_robot_animated.gif"  width="35%"> 



#### Create a Python Class for the Robot
You can find the complete code under [envs/robots/simple_robot.py](envs/robots/simple_robot.py).\
We follow the tutorial of [panda-gym](https://panda-gym.readthedocs.io/en/latest/custom/custom_robot.html).

The [tutorial](https://panda-gym.readthedocs.io/en/latest/custom/custom_robot.html) shows a simple class for a robot, which has only one joint. We extend this class to our robot, which has 5 joints. 

##### Parameters for initialization

We define our action space with 5 dimensions, which are the joint angles. The action space is defined as a Box, which is a continuous space. The action space is defined as:
```
n_action = 5   # 5 joints
action_space = spaces.Box(-1.0, 1.0, shape=(n_action,), dtype=np.float32)
```
Then we define our robot's physiqe by loading its URDF file:
```
body_name="simple_robot",
file_name="C:/Users/crypt/Dokumente/GitHub/panda-gym-tutorial/urdf/simple_robot.urdf",
```
The next thing is to define the joints which are used to control the robot. We load the joints as:
```
joint_indices=np.array([0, 1, 3, 5, 7]),
```
The joints must have the naming convention such as ``joint0'', ``joint1'', ``joint2'', etc. The joint indices are the indices of the joints in the URDF file. The first joint is the base joint, which is not used to control the robot. We only focus on the reach task, so we do not use the gripper and finger joints. Furthermore, the joint forces need to be defined. The joint forces are the maximum torque that can be applied to the joint. The joint forces are defined as:
```
joint_forces=np.array([40.0, 40.0, 40.0, 40.0, 40.0]),
```
Now, set the initial joint positions. The initial joint positions are defined as:
```
self.neutral_joint_values = np.array([0.00, 0.00, 0.00, 0.00, 0.00])
```
The End Effector is the last link of the robot. The End Effector is defined as:
```
self.ee_link = 8
```

##### Action Function
The action function is the function which is called to move the robot. The action function is defined as:
```
def action(self, action):
    self.set_joint_positions(action)
```
Some modifications are necessary to make everything work properly:
First, we ensure that the action lies within our action space:
```
action = np.clip(action, self.action_space.low, self.action_space.high)
```
Then we get the current joint positions in order to calculate the new joint positions:
```
current_arm_joint_angles = np.array([self.get_joint_angle(joint=i) for i in range(5)])
```
The new joint positions are calculated as:
```
target_arm_angles = current_arm_joint_angles + action
```
Now, set the new joint positions:
```
self.control_joints(target_angles=target_arm_angles)
```

##### Get Status Functions
The get status functions are used to get the current state of the robot. The get status functions are defined as:
```
    def get_obs(self) -> np.ndarray:
        # end-effector position and velocity
        ee_position = np.array(self.get_ee_position())
        ee_velocity = np.array(self.get_ee_velocity())
        # fingers opening

        observation = np.concatenate((ee_position, ee_velocity))
        return observation

    def reset(self) -> None:
        self.set_joint_neutral()

    def set_joint_neutral(self) -> None:
        """Set the robot to its neutral pose."""
        self.set_joint_angles(self.neutral_joint_values)

    def get_ee_position(self) -> np.ndarray:
        """Returns the position of the ned-effector as (x, y, z)"""
        return self.get_link_position(self.ee_link)

    def get_ee_velocity(self) -> np.ndarray:
        """Returns the velocity of the end-effector as (vx, vy, vz)"""
        return self.get_link_velocity(self.ee_link)
```

### Restricting the action space

The robot may have too much freedom of movement. This can result in the robot moving in a way that is not desired, e.g. the robot may move its arm in a way that it collides with the table. Or the algorithm may not be able to learn the task, because the action space is simply too large.

We can restrict the action space by defining a maximum joint angles inside the URDF file.

At first, load your URDF file into the[ Online URDF Modeler](https://mymodelrobot.appspot.com/5629499534213120). Here you can control the joint angles with sliders. 

Let's say we want to restrict the joint angles of the first joint to be between -90° and 90°. We can do this by setting the joint limits in the URDF file as:
```
<joint name="joint0" type="revolute">
	<joint name="joint0" type="revolute">  <!-- base -->
    	<parent link="link0"/>
    	<child link="link1"/>
    	<origin xyz="0 0 0.10" rpy="0 0 0"/>
    	<axis xyz="0 0 1"/>
    	<limit lower="-1.5708" upper="1.5708" effort="0" velocity="3"/>
</joint>
```
The following parameters are shown to solve the task but without much limitations:
```
joint0: -0.6 to 0.6
joint1:  0   to 1.6
joint3: -1.57 to 1.57
joint5: -1.57 to 1.57
joint7: -1.57 to 1.57
```

### Create a custom task.
In the [panda-gym tutorial](https://panda-gym.readthedocs.io/en/latest/custom/custom_task.html) you can find a detailed description on how to create a custom task. However, we will create a new 3D environment for the given task of reaching a target position, hence ``SimpleReach``.

#### Add Obstacles
We will add some obstacles to the environment and work in the file: [envs/tasks/simple_reach.py](envs/tasks/simple_reach.py).

The task has a function for creating the 3D pybullet environment:
```
    def _create_scene(self) -> None:
        self.sim.create_plane(z_offset=-0.4)
        self.sim.create_table(length=1.1, width=0.7, height=0.4, x_offset=-0.3)
        self.sim.create_sphere(
            body_name="target",
            radius=0.02,
            mass=0.0,
            ghost=True,
            position=np.zeros(3),
            rgba_color=np.array([0.1, 0.9, 0.1, 0.3]),
        )
```

The box has a position of (x, y, z) = (0.1, 0.15, 0.0) and a half extent of (x, y, z) = (0.05, 0.05, 0.2. This means that our box has a shape of 5 cm x 5 cm x 20 cm. In order to see the obstacle, we have to set the rgba color of the box as well. We choose a grey color with an alpha value of 1.0 for now.The box also needs a unique name for the 3d scene and we choose ``box0``. At last, we set the mass of the box to a very high value, so that the box does not move: ``mass=10e11``.

### Create Environment

In order to use our new robot, we have to define it in an environment.
So let's create a new environment. You can find the complete code under [envs/simple_env.py](envs/simple_env.py).

We initizalize our new robot with a base position with and X offset of 45 cm.\
As control type, we use the action space of joint angles, hence ``joints``.
The task is the reach task, which is defined in [envs/tasks/simple_reach.py](envs/tasks/simple_reach.py).

```
class SimpleEnv(RobotTaskEnv):
    def __init__(self, render_mode):
        sim = PyBullet(render_mode=render_mode)
        robot = SimpleRobot(sim, control_type="joints", base_position=[-0.45, 0, 0])
        task = Reach(sim, robot.get_ee_position)
        super().__init__(robot, task)
```

#### Test the new Robot in the new Environment
We are now ready to test our new robot in the new environment with an obstacle.

In [ ]:
#############################################
# Task: Panda Pick and Place with Obstacles #
#############################################

import gymnasium as gym
import panda_gym
from envs.simple_env import SimpleEnv
from tqdm import tqdm
from numpngw import write_apng

# please choose one of the following render modes
render_mode = 'human'       # opens OpenGL window for rendering
#render_mode = 'rgb_array'  # returns RGB image as numpy array, if the render should be stored as animated png

# create environment and activate rendering
if render_mode == 'rgb_array':
    images = [] # array to store images

env = SimpleEnv(render_mode=render_mode)

# define low frame rate for rendering to reduce computational load
env.metadata['render_fps'] = 24

# reset environment and get initial observation (either state of the joints or the end effector position)
observation, info = env.reset()
# define maximum number of episodes
max_steps = 10000

# run simulation 
for step in tqdm(range(max_steps)):
    # take action as defined by our policy
    action = env.action_space.sample()
    
    # execute action and get new observation, reward, termination flag and additional info
    observation, reward, terminated, truncated, info = env.step(action)

    # add each image to our array for each step to create an animation afterwards
    if render_mode == 'rgb_array':
        images.append(env.render())

    # when the episode is terminated, reset the environment
    if terminated or truncated:
        observation, info = env.reset()

env.close()

if render_mode == 'rgb_array':
    # save animation
    print("Saving animation...")
    write_apng('images/simple_reach_w_obstacles.png', images, delay = 5)
print("finished")

#### Animation of the new simple robot in an environment with an obstacle

<img src="images/simple_reach_w_obstacles.png" width="35%"> 

As you can see, the robot is able to reach the target position even with an obstacle in the scene.

### Summary

1. At first we created a new robot model with the Universal Robot Description Format (URDF).
    * [urdf/simple_robot.urdf](urdf/simple_robot.urdf).
2. Then we imported the URDF file and parameterized the robot correctly.
In here, we also defined the action space and functions for the status of the robot.\
We implemented how the robot takes the new action and how the observation is created.
    * [robots/simple_robot.py](robots/simple_robot.py).
3. After that, we restricted the action space (joint angles) to a reasonable range to make the task easier to solve.
    * [urdf/simple_robot.urdf](urdf/simple_robot.urdf).
4. A new task is defined to reach a target position, but with obstacles in the scene.
    * [envs/tasks/simple_reach.py](envs/tasks/simple_reach.py).
5. Finally, this new task and the new robot are combined in a new environment.
    * [envs/simple_env.py](envs/simple_env.py).